In [44]:
from gurobipy import *
import pandas as pd

transport_cost = 1 #eur per km
M = 100000
demand = 100 #per week
dists = pd.read_csv('cities_dist.csv')

In [45]:
def get_dist_model(dists, transport_cost, fixed_cost, M, demand):
    cities = list(dists)[1:]
    
    model = Model('Distribution')
    model.Params.OutputFlag = 0
    X = model.addVars(cities, vtype=GRB.BINARY, name="DC_opened")
    A = model.addVars(cities, cities, vtype=GRB.CONTINUOUS, name="Amount") 

    model.addConstrs((A[DC, DA] >= 0 for DC in cities for DA in cities), name = "Min_amount")
    model.addConstrs(((quicksum(A[DC, DA] for DC in cities) >= demand) for DA in cities), name="Demand")
    model.addConstrs(((quicksum(A[DC, DA] for DA in cities) <= M * X[DC]) for DC in cities), name="Zero_demand_if_DC_not_openned")    
    obj = quicksum(X[DC] * fixed_cost for DC in cities) + quicksum(quicksum(transport_cost * dists[DC][cities.index(DA)] * A[DC, DA] for DC in cities) for DA in cities)
    model.setObjective(obj, GRB.MINIMIZE)
    model.optimize()
    if model.Status == GRB.OPTIMAL:
    	opened_cities = [city for city, value in X.iteritems() if value.X > 0]
    	for v in model.getVars():
            if v.X != 0:
            	pass    
            #	print(v.x)
            #print("%s %f" % (v.Varname, v.X))
    else:
    	opened_cities = ['Not optimal']
    return opened_cities
   
prev = []
for fixed_cost_tmp in range(100000,800000,10000):
    DC = get_dist_model(dists, transport_cost, fixed_cost_tmp, M, demand)
    if DC != prev:
        print("Fixed cost: %d --> %s" % (fixed_cost_tmp, DC))
    prev = DC

Fixed cost: 100000 --> ['Athens', 'Brussels', 'Hamburg', 'Madrid', 'Munich', 'Naples', 'Turin']
Fixed cost: 110000 --> ['Athens', 'Brussels', 'Hamburg', 'Madrid', 'Munich', 'Rome', 'Turin']
Fixed cost: 130000 --> ['Antwerp', 'Athens', 'Genoa', 'Madrid', 'Prague', 'Strasbourg']
Fixed cost: 150000 --> ['Antwerp', 'Athens', 'Madrid', 'Munich', 'Turin']
Fixed cost: 230000 --> ['Antwerp', 'Madrid', 'Munich', 'Turin']
Fixed cost: 260000 --> ['Antwerp', 'Madrid', 'Milan']
Fixed cost: 350000 --> ['Antwerp', 'Turin']
Fixed cost: 740000 --> ['Strasbourg']


In [3]:
from ipywidgets import widgets, interact

def get_dist_model_interact(fixed_cost):
    print(get_dist_model(dists, transport_cost, fixed_cost, M, demand))
    
interact(get_dist_model_interact, fixed_cost=widgets.IntSlider(min=1,max=800000,step=20000,value=100000))

['Antwerp', 'Turin']


In [2]:
import Basemap

ImportError: No module named 'Basemap'